In [ ]:
import sagemaker
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket ='mobbucketsk'
print("Using bucket" + bucket)

In [ ]:
df = pd.read_csv("mob_price.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df['price_range'].value_counts(normalize=True)

In [ ]:
features = list(df.columns)
features

In [ ]:
label = features.pop(-1)

In [ ]:
label

In [ ]:
x= df[features]
y = df[label]

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
y.value_counts()

In [ ]:
X_train,X_test,y_train, y_test = train_test_split(x,y,test_size=0.15,random_state=0)

In [ ]:
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test

In [ ]:
print(trainX.shape)
print(testX.shape)

In [ ]:
trainX.to_csv("train-V-1.csv",index = False)
testX.to_csv("test-V-1.csv",index = False)

In [ ]:
sk_prefix = "sagemaker/mobi_price_classification/sklearncontainer"
trainpath = sess.upload_data(
    path="train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)


In [ ]:
testpath = sess.upload_data(
    path="test-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

In [ ]:
print(testpath)
print(trainpath)

In [ ]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  accuracy_score,classification_report,confusion_matrix,precision_score
import sklearn
import boto3
import pathlib
from io import  StringIO
import argparse
import joblib
import os
import numpy as np
import pandas as pd



def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ == "__main__":
    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    #hyperparameters sent by the client are passed as command-line arguments
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state",type=int, default=0)

    #data, model, and output directories

    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")


    args, _ = parser.parse_known_args()

    print("SKLearn Version:", sklearn.__version__)
    print("Joblib Version:", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))


    features = list(train_df.columns)
    label = features.pop(-1)


    print("Building training and testing datasets")

    print()

    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]


    print('Column order:')
    print(features)
    print()

    print("Label column is:",label)
    print()


    print("Data Shape:")
    print()

    print("----- SHAPE OF TRAINING DATA (85%)-------")
    Print("X_train.shape")
    print("y_train.shape")

    print("-----SHAPE OF TESTING DATA (15%)--------")
    print("X_test.shape")
    print("y_test.shape")


    print("Training RandomForest Model......")
    print()
    model = RandomForestClassifier(n_estimators=args.n_estimators,random_state=args.random_state)
    model.fit(X_train, y_train)
    print()


    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at" + model_path)
    print()


    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)



    print()
    print("----- METRICS RESULTS FOR TESTING DATA----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc )
    print('[TESTING] Testing report: ')
    print(test_rep)



In [ ]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::084719916966:role/service-role/AmazonSageMaker-ExecutionRole-20250913T134470",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [ ]:
#launching training job, with asynchronous call
sklearn_estimator.fit({"train":trainpath,"test":testpath}, wait=True)



In [ ]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at" + artifact)

In [ ]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name = model_name,
    model_data=artifact,
    role="arn:aws:iam::084719916966:role/service-role/AmazonSageMaker-ExecutionRole-20250913T134470",
    entry_point="script.py",
    framework_version="FRAMEWORK_VERSION",
)

In [ ]:
model_name

In [ ]:
#endpoints deployment

endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndPointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)

In [ ]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)